<a href="https://colab.research.google.com/github/dapopov-st/ExperimentsWithLanguageModels/blob/main/Fine_tune_Llama_2_13b_on_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2-13B on SQL

Built starting from  [Colab NB](https://colab.research.google.com/drive/1PEQyJO1-f6j0S_XJ8DV50NkpzasXkrzd?usp=sharing).

This notebook runs on a A100 GPU on Colab. (Last update: 24 Aug 2023)


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 118.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


In [112]:
import os
import torch
from datasets import load_dataset
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [4]:
from huggingface_hub import login
login()

In [105]:
from google.colab import drive
drive.mount('/content/drive')

output_dir = '/content/drive/MyDrive/SQL/sql_output_dir'
logging_dir = '/content/drive/MyDrive/SQL/sql_logging_dir'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()

In [107]:
clean_mem()

### Axolotl approach: got odd cudnn errors

In [63]:
import accelerate

In [69]:
%%bash
ls '/content/drive/MyDrive/SQL/'
accelerate launch -m axolotl.cli.train "/content/drive/MyDrive/SQL/sql.yml"

sql_logging_dir
sql_output_dir
sql.yml


### Step-by-step setup

In [126]:
# The model that you want to train from the Hugging Face hub
#model_name = "NousResearch/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-hf"

# The instruction dataset to use
#dataset_name = "mlabonne/guanaco-llama2-1k"
dataset_name = "knowrohit07/know_sql"

# Fine-tuned model name
new_model = "llama-2-13b-sql"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 32 # TODO: changed from 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.05 #TODO: changed from .1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = output_dir

# Number of training epochs
num_train_epochs = 5

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [129]:
# Define the formatting function
def formatting_func(example): #Worked, but result wasn't great: did not use GROUP BY in SQL
    text = f"### Question: {example['question']}\n ### Context: {example['context']}\n ### Answer: {example['answer']}"
    #print(text.split('\n'))
    return text.split('\n')


In [30]:
# Load dataset (you can process it here)
#dataset = load_dataset(dataset_name, split="train")
#dataset = load_dataset(dataset_name, split="validation") #TODO: works,trains super quikcly, guess grabbed small subset
dataset=load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')['train']

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = LlamaForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1



Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.unk_token = "<unk>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


In [31]:
# Set training parameters.  Training for 1 epoch.
training_arguments = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)



# Train the model
trainer.train()


# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/78562 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.4418, 'learning_rate': 0.000131930153013598, 'epoch': 0.42}
{'loss': 1.2081, 'learning_rate': 1.4314282383241096e-05, 'epoch': 0.83}
{'train_runtime': 219.3233, 'train_samples_per_second': 1.081, 'train_steps_per_second': 0.274, 'train_loss': 1.328073501586914, 'epoch': 1.0}


In [75]:
# Set training parameters: Training for 5 epochs
training_arguments = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)



# Train the model
trainer.train()


# Save trained model
trainer.model.save_pretrained(new_model)

Map:   0%|          | 0/78562 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.4518, 'learning_rate': 0.00019851186053243666, 'epoch': 0.42}
{'loss': 1.1896, 'learning_rate': 0.00019036283606085053, 'epoch': 0.83}
{'loss': 1.1934, 'learning_rate': 0.00017567128158176953, 'epoch': 1.25}
{'loss': 1.218, 'learning_rate': 0.000155500908021347, 'epoch': 1.67}
{'loss': 1.1128, 'learning_rate': 0.00013131210861240026, 'epoch': 2.08}
{'loss': 1.0871, 'learning_rate': 0.00010485622221144484, 'epoch': 2.5}
{'loss': 1.0573, 'learning_rate': 7.804873131325954e-05, 'epoch': 2.92}
{'loss': 1.0316, 'learning_rate': 5.283057559252341e-05, 'epoch': 3.33}
{'loss': 0.9779, 'learning_rate': 3.102762227218957e-05, 'epoch': 3.75}
{'loss': 1.0043, 'learning_rate': 1.4218468069322578e-05, 'epoch': 4.17}
{'loss': 1.0098, 'learning_rate': 3.620144238882206e-06, 'epoch': 4.58}
{'loss': 0.9497, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 1097.6116, 'train_samples_per_second': 1.08, 'train_steps_per_second': 0.273, 'train_loss': 1.106935551961263, 'epoch': 5.0}


In [28]:
ds = load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [131]:
trn = ds['train']
trn[3]



{'question': 'What are the hosts of competitions whose theme is not "Aliens"?',
 'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)',
 'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'"}

In [132]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst



{'question': 'Get the count of competition hosts by theme.',
 'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)',
 'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'"}

In [136]:
#def sql_prompt(d): return fmt.format(d["context"], d["question"], d['answer'])
def sql_prompt(example): return f"### Question: {example['question']}\n ### Context: {example['context']}\n ### Answer: {example['answer']}"

In [137]:
print(sql_prompt(tst))

### Question: Get the count of competition hosts by theme.
 ### Context: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
 ### Answer: SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'


In [138]:
toks = tokenizer(sql_prompt(tst), return_tensors="pt")

In [139]:
res = trainer.model.generate(**toks.to("cuda"), max_new_tokens=250).to('cpu')

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [151]:
for elt in tokenizer.batch_decode(res):
  print(elt)


<s> ### Question: Get the count of competition hosts by theme.
 ### Context: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
 ### Answer: SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'

### Question: Get the count of competition hosts by theme and the count of competitions by theme.
 ### Context: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
 ### Answer: SELECT Hosts, COUNT(Theme) FROM farm_competition GROUP BY Hosts

### Question: Get the count of competition hosts by theme and the count of competitions by theme.
 ### Context: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
 ### Answer: SELECT Hosts, COUNT(Theme) FROM farm_competition GROUP BY Hosts

### Question: Get the count of competition hosts by theme and the count of competitions by theme.
 ### Context: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
 ### Answer: SELECT Hosts, COUNT(Theme) FROM farm_competition GROUP BY Hosts

### Question: Get the count of compe

In [152]:
trainer.save_model(output_dir)

###Possible TODOs:
- Evaluate formally
- Train a Mistral version
- Comapare to other models (GPT4 and Mistral's model trained on SQL): Expect Mistral 7B to do better according to https://dev.to/ananddas/mistral-7b-beats-llama-2-13b-on-all-benchmarks-55j2
- Train a larger Mistral model then

In [37]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [159]:
#!model.push_to_hub(trainer.model, use_temp_dir=False)
#!tokenizer.push_to_hub(trainer.model, use_temp_dir=False)